In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
dataset = pd.read_csv("insurance_pre.csv")
print(dataset)

      age     sex     bmi  children smoker      charges
0      19  female  27.900         0    yes  16884.92400
1      18    male  33.770         1     no   1725.55230
2      28    male  33.000         3     no   4449.46200
3      33    male  22.705         0     no  21984.47061
4      32    male  28.880         0     no   3866.85520
...   ...     ...     ...       ...    ...          ...
1333   50    male  30.970         3     no  10600.54830
1334   18  female  31.920         0     no   2205.98080
1335   18  female  36.850         0     no   1629.83350
1336   21  female  25.800         0     no   2007.94500
1337   61  female  29.070         0    yes  29141.36030

[1338 rows x 6 columns]


In [23]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [25]:
print(dataset)

      age     bmi  children      charges  sex_male  smoker_yes
0      19  27.900         0  16884.92400     False        True
1      18  33.770         1   1725.55230      True       False
2      28  33.000         3   4449.46200      True       False
3      33  22.705         0  21984.47061      True       False
4      32  28.880         0   3866.85520      True       False
...   ...     ...       ...          ...       ...         ...
1333   50  30.970         3  10600.54830      True       False
1334   18  31.920         0   2205.98080     False       False
1335   18  36.850         0   1629.83350     False       False
1336   21  25.800         0   2007.94500     False       False
1337   61  29.070         0  29141.36030     False        True

[1338 rows x 6 columns]


In [33]:
independent_input=dataset[['age','bmi','children','sex_male','smoker_yes']]
independent_input

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,False,True
1,18,33.770,1,True,False
2,28,33.000,3,True,False
3,33,22.705,0,True,False
4,32,28.880,0,True,False
...,...,...,...,...,...
1333,50,30.970,3,True,False
1334,18,31.920,0,False,False
1335,18,36.850,0,False,False
1336,21,25.800,0,False,False


In [29]:
dependent_output=dataset[["charges"]]
dependent_output

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [35]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(independent_input,dependent_output,test_size=1/3,random_state=0)


In [37]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
parameter_grid={'kernel':['rbf','linear','sigmoid','poly'],
               'C':[10,100,1000,2000,3000],
               'gamma':['auto','scale']}
grid=GridSearchCV(SVR(),parameter_grid, refit=True ,verbose=3,n_jobs=-1)
grid .fit(x_train,y_train)



Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'linear', 'sigmoid', 'poly']},
             verbose=3)

In [47]:
re=grid.cv_results_ #cv_results_ is function parameter
print("The R_Score value for best parameter {}".format(grid.best_params_))

The R_Score value for best parameter {'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}


In [53]:
table=pd.DataFrame.from_dict(re)
print(table)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  param_C  \
0        0.060347      0.015288         0.022206        0.004353       10   
1        0.032050      0.004829         0.010153        0.004530       10   
2        0.051091      0.002884         0.008888        0.001105       10   
3        0.033888      0.002007         0.005953        0.003171       10   
4        0.040843      0.003640         0.030729        0.001980       10   
5        0.033167      0.002895         0.002951        0.002848       10   
6        0.048536      0.003043         0.009464        0.004112       10   
7        0.031419      0.001048         0.004589        0.003628       10   
8        0.043333      0.003258         0.022410        0.004796      100   
9        0.038317      0.009706         0.006912        0.003743      100   
10       0.047811      0.002515         0.007631        0.004315      100   
11       0.035971      0.003996         0.004715        0.003396      100   

In [55]:
age_input=float(input("Age"))
bmi_input=float(input("BMI"))
children_input=float(input("children"))
sex_male_input=int(input("Sex male 0 or 1"))
smoker_yes_input=int(input("smoker yes 0 or 1"))


Age 20
BMI 35
children 2
Sex male 0 or 1 0
smoker yes 0 or 1 1


In [59]:
Future_prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future prediction {}".format(Future_prediction))

Future prediction [62587.53934691]


In [61]:
import pickle
filename="Finalised_grid_search.sav"
pickle.dump(grid,open(filename,'wb'))

In [63]:
loaded_model=pickle.load(open("Finalised_grid_search.sav",'rb'))

In [65]:
result=loaded_model.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print(result)

[62587.53934691]
